# Study: Cutouts vs. Steps Per Frame

This is an executable notebook. To open in colab, click the "Launch" icon above (the rocket ship). Once in colab, run the following commands in a new cell to install pytti:

```
!git clone --recurse-submodules -j8 https://github.com/pytti-tools/pytti-core
%pip install ./pytti-core/vendor/AdaBins
%pip install ./pytti-core/vendor/CLIP
%pip install ./pytti-core/vendor/GMA
%pip install ./pytti-core/vendor/taming-transformers
%pip install ./pytti-core
!python -m pytti.warmup
!touch config/conf/empty.yaml
```

## Specify experiment parameters:

In [ ]:
cross_product__quality0 = (
    ("cutouts", (10, 40, 160)),
    ("steps_per_frame", (20, 80, 160)) # would be way more efficient to just use save_every
)

invariants0 = {
    'scenes':'"portrait of a man, oil on canvas"',
    'image_model':'VQGAN',
    'conf':'empty', # I should just not require conf here...
    'seed':12345,
    }

# variable imputation doesn't seem to work in the overrides
map_kv = (
    ('steps_per_frame', ('steps_per_scene','pre_animation_steps', 'display_every', 'save_every')),
)


In [ ]:

# this is useful enough that I should just ship it with pytti

from copy import deepcopy
from itertools import (
    product, 
    combinations,
)
from hydra import initialize, compose
from loguru import logger
from pytti.workhorse import _main as render_frames

def build_experiment_parameterizations(
    cross_product,
    invariants,
    map_kv,
):
    kargs = []
    NAME, VALUE = 0, 1
    for param0, param1 in combinations(cross_product, 2):
        p0_name, p1_name = param0[NAME], param1[NAME]
        for p0_val, p1_val in product(param0[VALUE], param1[VALUE]):
            kw = deepcopy(invariants)
            kw.update({
                p0_name:p0_val,
                p1_name:p1_val,
                'file_namespace':f"matrix_{p0_name}-{p0_val}_{p1_name}-{p1_val}",
                })
            # map in "variable imputations"
            for k0, krest in map_kv:
                for k1 in krest:
                    kw[k1] = kw[k0]
            kargs.append(kw)
    kws = [[f"{k}={v}" for k,v in kw.items()] for kw in kargs]
    return kargs, kws

def run_experiment_matrix(
    kws,
    CONFIG_BASE_PATH = "config",
    CONFIG_DEFAULTS = "default.yaml",
):
    # https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
    # https://omegaconf.readthedocs.io/
    # https://hydra.cc/docs/intro/
    with initialize(config_path=CONFIG_BASE_PATH):

        for k in kws:
            logger.debug(k)
            cfg = compose(config_name=CONFIG_DEFAULTS, 
                        overrides=k)
            render_frames(cfg)

            

In [ ]:
%%capture
kargs, kws = build_experiment_parameterizations(
    cross_product__quality0,
    invariants0,
    map_kv,
)

run_experiment_matrix(kws)

In [ ]:
# https://pytorch.org/vision/master/auto_examples/plot_visualization_utils.html#visualizing-a-grid-of-images
# sphinx_gallery_thumbnail_path = "../../gallery/assets/visualization_utils_thumbnail2.png"
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from torchvision.io import read_image
import torchvision.transforms.functional as F
from torchvision.utils import make_grid


plt.rcParams["savefig.bbox"] = 'tight'
plt.rcParams['figure.figsize'] = 20,20

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    return fix, axs


images = []
for k in kargs:
    fpath = Path("images_out") / k['file_namespace'] / f"{k['file_namespace']}_1.png"
    images.append(read_image(str(fpath)))

nr = len(cross_product__quality0[0][-1])
grid = make_grid(images, nrow=nr)
fix, axs = show(grid)

ax0_name, ax1_name = cross_product__quality0[0][0], cross_product__quality0[1][0]
fix.savefig(f"TestMatrix_{ax0_name}_{ax1_name}.png")

# to do: 
# * label axes and values
# * track and report runtimes for each experiment
# * track and report runtime of notebook